# Latency and Tick

## Python Imports

In [1]:
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt

In [2]:
from scipy.optimize import minimize

In [3]:
from functools import partial

## Tick Imports

In [4]:
from tick.hawkes import SimuHawkes, SimuHawkesMulti
from tick.base import TimeFunction
from tick.hawkes import HawkesKernelTimeFunc, HawkesKernelExp

In [5]:
import hawklat as hl

## Estimation of slowly decreasing Hawkes kernels: application to high-frequency order book dynamics

In [6]:
from tick.dataset import fetch_hawkes_bund_data

In [7]:
timestamps_list = fetch_hawkes_bund_data()

/Users/marcoscscarreira/Anaconda/anaconda3/envs/mcsc/lib/python3.7/site-packages/tick/dataset/download_helper.py:216: UserWarning: TICK_DATASETS environment variable was not set. Saving dataset to the default location ~/tick_datasets
  .format(_TICK_HOME_ENV, _TICK_DEFAULT_HOME))


In [8]:
[len(timestamps_list), len(timestamps_list[0]), len(timestamps_list[0][0])]

[20, 4, 5295]

In [9]:
bunds_M = 4

In [21]:
bund_bounds = [(1e-4, 1e+2)] * bunds_M + [(1e-2, 1e+6)] * (bunds_M ** 2) + [(1e-2, 1e+6)] * (bunds_M ** 2)

In [22]:
bund_bounds_de = [(1e-4, 1e+2)] * bunds_M + [(1e-2, 1e+6)] * (bunds_M ** 2) + [(1e-2, 1e+6)] * (bunds_M ** 2)

In [23]:
bund_x0 = np.array([1e-1] * bunds_M + [1e-2] * (bunds_M ** 2) + [1e-1] * (bunds_M ** 2))

In [13]:
lat_bund = 250e-6

In [14]:
def show_results(result):
    λ0s = np.array([result[j][0] for j in range(bunds_M)])
    αs = np.array([result[j][1:bunds_M + 1] for j in range(bunds_M)])
    βs = np.array([result[j][bunds_M + 1:] for j in range(bunds_M)])
    return [λ0s, αs, βs]

## Day 0

In [24]:
%%time

bund_result = hl.findθlatm(timestamps_list[0], bounds=bund_bounds, x0=bund_x0, τ=lat_bund, method='L-BFGS-B', print_m=True)

m = 0
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-59798.875207009
m = 1
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-61534.937831638614
m = 2
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-43201.05394931278
m = 3
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-43718.87387218821
CPU times: user 11min 5s, sys: 1.35 s, total: 11min 7s
Wall time: 11min 8s


In [25]:
show_results(bund_result)

[array([0.02972414, 0.02844437, 0.03815923, 0.03680496]),
 array([[2.56067439e+02, 1.48270015e+03, 3.92679722e+02, 1.34741815e+02],
        [8.69680580e+02, 2.45056916e+02, 6.04519949e+00, 2.54739087e+02],
        [1.00000000e-02, 4.76962954e+02, 3.67049951e+00, 2.25334974e-01],
        [5.22664365e+02, 1.00000000e-02, 7.29361595e-01, 2.36907220e+00]]),
 array([[1.49796954e+03, 2.73586157e+03, 3.01797791e+03, 3.30100497e+03],
        [2.42470812e+03, 1.18543895e+03, 1.84467190e+03, 2.43486326e+03],
        [1.68315034e-02, 9.62026464e+02, 7.98153701e+00, 1.11883119e+01],
        [9.28208211e+02, 1.80279850e-02, 9.94812385e+01, 4.08581153e+00]])]

#### No latency on inference leads to a bad estimate

In [28]:
%%time

bund_result_0 = hl.findθlatm(timestamps_list[0], bounds=bund_bounds, x0=bund_x0, τ=0, method='L-BFGS-B', print_m=True)

m = 0
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-107193.7906956965
m = 1
b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
-119805.1232022354
m = 2
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-160410.86954295423
m = 3
b'ABNORMAL_TERMINATION_IN_LNSRCH'
-183697.3056675103
CPU times: user 5min 24s, sys: 672 ms, total: 5min 24s
Wall time: 5min 25s


In [29]:
show_results(bund_result_0)

[array([0.0001, 0.0001, 0.0001, 0.0001]),
 array([[6.17233544e+04, 1.00000000e-02, 1.00000000e-02, 1.00000000e-02],
        [1.00000000e-02, 5.97702943e+05, 1.00000000e-02, 1.00000000e-02],
        [1.00000000e-02, 1.00000000e-02, 1.08458364e+04, 1.00000000e-02],
        [1.00000000e-02, 1.00000000e-02, 1.00000000e-02, 1.03508446e+04]]),
 array([[ 206238.75328963,   37390.75417917,   49433.78045621,
           61000.69107067],
        [ 280732.4079973 , 1000000.        ,  422978.97555262,
          489973.19951637],
        [   2976.02218765,    3309.77973189,   24189.50757544,
            5163.92936526],
        [   2385.19871058,    2363.18078309,    3838.2692137 ,
           23102.30240808]])]

## Day 0 - Differential Evolution

In [26]:
%%time

bund_result = hl.findθlatmde(timestamps_list[0], bounds=bund_bounds_de, τ=lat_bund, maxiter=300, popsize=20, tol=1e-2,
                             print_m=True, disp=False)

m = 0
Optimization terminated successfully.
-60985.36461429661
m = 1
Optimization terminated successfully.
-62022.37044688074
m = 2
Optimization terminated successfully.
-42804.27917211133
m = 3
Optimization terminated successfully.
-43847.610296090046
CPU times: user 4h 43min 52s, sys: 27.3 s, total: 4h 44min 19s
Wall time: 4h 44min 38s


In [27]:
show_results(bund_result)

[array([0.0270867 , 0.02977512, 0.09519626, 0.08616202]),
 array([[1.17208723e+03, 9.40919301e+01, 4.14563326e+02, 5.48499180e+03],
        [2.85131621e+02, 8.52502714e+02, 1.15877792e+03, 3.64604355e+02],
        [2.66037192e+03, 2.63089766e+02, 3.27099926e+00, 6.51009868e+01],
        [2.79965303e+02, 3.56804952e+02, 1.15731209e+02, 2.12751995e+00]]),
 array([[6.60624850e+03, 3.98170358e+02, 3.12501705e+03, 9.48762173e+05],
        [1.60831842e+03, 4.08627437e+03, 3.14645389e+05, 3.11796379e+03],
        [6.72975846e+05, 1.09652246e+03, 6.58618594e+00, 3.63866762e+05],
        [1.19437160e+03, 1.67632425e+04, 3.88331719e+05, 3.45673500e+00]])]

#### No latency on inference leads to a bad estimate

In [30]:
%%time

bund_result_0 = hl.findθlatmde(timestamps_list[0], bounds=bund_bounds_de, τ=0, maxiter=300, popsize=20, tol=1e-2,
                             print_m=True, disp=True)

differential_evolution step 1: f(x)= -83268.2
differential_evolution step 2: f(x)= -83268.2
differential_evolution step 3: f(x)= -83268.2
differential_evolution step 4: f(x)= -93779.5
differential_evolution step 5: f(x)= -101817
differential_evolution step 6: f(x)= -106593
differential_evolution step 7: f(x)= -109199
differential_evolution step 8: f(x)= -112258
differential_evolution step 9: f(x)= -112573
differential_evolution step 10: f(x)= -112573
differential_evolution step 11: f(x)= -112573
differential_evolution step 12: f(x)= -114014
differential_evolution step 13: f(x)= -114014
differential_evolution step 14: f(x)= -114014
differential_evolution step 15: f(x)= -114014
differential_evolution step 16: f(x)= -114014
differential_evolution step 17: f(x)= -114798
differential_evolution step 18: f(x)= -114798
differential_evolution step 19: f(x)= -114798
differential_evolution step 20: f(x)= -115625
differential_evolution step 21: f(x)= -115625
differential_evolution step 22: f(x)= -

In [31]:
show_results(bund_result_0)

[array([0.0001, 0.0001, 0.0001, 0.0001]),
 array([[7.67208313e+05, 1.03501731e+04, 1.12484909e+03, 3.08018888e+03],
        [6.55957423e+03, 9.22793417e+05, 2.55311487e+03, 6.58212033e+02],
        [1.03558841e+04, 1.57330988e+04, 9.43320250e+05, 1.00000000e-02],
        [5.71627283e+04, 1.36659038e+04, 3.30510346e+04, 9.64034125e+05]]),
 array([[990354.90317211, 744881.42381639, 679496.97985725,
         617124.33580757],
        [670904.35362485, 982182.6903935 , 740771.78833091,
         577190.82428488],
        [328083.58973268, 771721.57156598, 996825.54932178,
         440961.6536225 ],
        [788596.07611558, 566419.37530591, 983876.13535997,
         997155.00282689]])]

## Day 1

In [34]:
%%time

bund_result_1 = hl.findθlatm(timestamps_list[1], bounds=bund_bounds, x0=bund_x0, τ=lat_bund, method='L-BFGS-B', print_m=True)

m = 0
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-62128.00642575798
m = 1
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-61141.045042844096
m = 2
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-42679.41453769193
m = 3
b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
-46673.37636351674
CPU times: user 10min 18s, sys: 1.6 s, total: 10min 19s
Wall time: 10min 20s


In [35]:
show_results(bund_result_1)

[array([0.0244582 , 0.02547934, 0.09384817, 0.03164681]),
 array([[3.90290807e+02, 2.07300557e+03, 1.57715346e+02, 9.65421606e+00],
        [1.08640257e+03, 3.07889342e+02, 8.50123387e+00, 5.73768838e+02],
        [1.61980771e-02, 3.73056667e+00, 1.09358109e+01, 7.90958783e-01],
        [2.16228456e+02, 8.40546255e-01, 1.00000000e-02, 2.26396886e+00]]),
 array([[1.92118826e+03, 2.82984262e+03, 2.26299944e+03, 2.86753615e+03],
        [2.27425608e+03, 1.75910192e+03, 2.26992439e+03, 3.20336403e+03],
        [3.35207719e-02, 1.70617102e+01, 4.07363439e+01, 9.13102908e+00],
        [1.01142357e+03, 1.99815951e+01, 3.76613532e-02, 3.63471125e+00]])]

## Day 1 - Differential Evolution

In [36]:
%%time

bund_result_1 = hl.findθlatmde(timestamps_list[1], bounds=bund_bounds_de, τ=lat_bund, maxiter=300, popsize=25, tol=1e-2,
                             print_m=True, disp=False)

m = 0
Optimization terminated successfully.
-63540.322481019975
m = 1
Optimization terminated successfully.
-62283.82358426356
m = 2
Optimization terminated successfully.
-43441.653869177404
m = 3
Optimization terminated successfully.
-45783.290700785976
CPU times: user 7h 58min 42s, sys: 31.9 s, total: 7h 59min 14s
Wall time: 7h 59min 35s


In [37]:
show_results(bund_result_1)

[array([0.0251267 , 0.02355387, 0.10307676, 0.08139396]),
 array([[7.98338311e+02, 4.39133128e+02, 3.50147088e+02, 1.36807204e+01],
        [1.15912831e+02, 1.26585129e+03, 2.91379268e+03, 2.18785981e+02],
        [2.24532289e+03, 3.03337271e+02, 4.70820985e+00, 4.01813081e+02],
        [2.24396859e+02, 1.22961648e+03, 2.64083161e+02, 1.87123506e+00]]),
 array([[3.90332492e+03, 2.03607169e+03, 2.80825151e+03, 4.02579657e+05],
        [5.11846799e+02, 4.48891129e+03, 5.09490336e+05, 2.54271332e+03],
        [6.78551596e+05, 1.19547326e+03, 9.58937481e+00, 6.21292719e+05],
        [8.89620385e+02, 7.66305179e+05, 2.71455947e+05, 2.77027597e+00]])]